# CS5830 Project 5: Naive Bayes

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from ucimlrepo import fetch_ucirepo # Dataset
from sklearn.preprocessing import StandardScaler # Data standardization
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

# Models
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB

## Fetch and Clean the Data

In [ ]:
adult = fetch_ucirepo(id=2)
X = adult.data.features
y = adult.data.targets

# Convert <=50k to 0 and >50k to 1 (the dataset also has a . at the end of some values, which we need to account for)
print(y['income'].unique())
y['income'] = y['income'].map({'<=50K': 0, '>50K': 1, '<=50K.': 0, '>50K.': 1})
print(y['income'].unique())

df = pd.concat([X, y], axis=1)

df = df.replace('?', np.nan)
df = df.dropna()
df.head()

In [ ]:
"""
Declare categorical and numerical features
"""

adult_feats = adult.variables
adult_feats = adult_feats[adult_feats["role"] != "Target"]
display(adult_feats)

categorical_feats = adult_feats[(adult_feats["type"] == "Categorical") | (adult_feats["type"] == "Binary")]['name'].tolist()
numerical_feats = adult_feats[adult_feats["type"] == "Integer"]['name'].tolist()

print(f'Categorical: {categorical_feats}')
print(f'Numerical: {numerical_feats}')

In [ ]:
"""
Prepare categorical and numerical features
"""

# one-hot encode the categorical features
df = pd.get_dummies(df, columns=categorical_feats, drop_first=True)

X = df.drop('income', axis=1)
y = df['income']

X_train, X_test, y_train, y_test = train_test_split(X, y)

# Scale the numerical features
scaler = StandardScaler()
X_train[numerical_feats] = scaler.fit_transform(X_train[numerical_feats])
X_test[numerical_feats] = scaler.transform(X_test[numerical_feats])

X_train

In [ ]:
"""
Now with all features cleaned, we can perform feature selection
"""

n_features = 20
kbest = SelectKBest(k=n_features)
X_train = kbest.fit_transform(X_train, y_train)
X_test = kbest.transform(X_test)

## Naive Bayes Classification

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_gnb = gnb.predict(X_test)
gnb_scores = precision_recall_fscore_support(y_test, y_pred_gnb, average='binary')

print(f'Naive Bayes (Gaussian) Scores: precision={gnb_scores[0]}, recall={gnb_scores[1]}, f1={gnb_scores[2]}')